# **Description**:

This script was developed to replicate Table 4 on the manuscript "Relationship Between Environmental Values and the Evaluation of Watershed Conservation Plans that Benefit the Community Versus the Individual." It is designed to be executed in R.

Use R Colab version for replication in a cloud-based environment using the R kernel.

To prepare the final Figure for presentation in the manuscript, the resulting values were formatted in Excel.

## Install libraries

Installing libraries in R-colab may take some minutes

In [1]:
# Load the necessary library for data manipulation.
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




### Set working directory

No needed in R-Colab. For R-colab, upload the required file to session storage

In [ ]:
### Set the working director
setwd("E:/OSU_PC_D_drive/research/thesis_papers/Paper_02_DM_Human_values/analysis/002_obj2c_gradient_of_ratings_PPM")

### **Load data**

In [3]:
# 1. Load the raw dataset (CSV format)
df <- read.csv("Rating_of_participants_to_the_suggested_plans.csv")
head(df)#; names(df)

,Particip,pid,SP1,SP2,SP3,SP4,SP5,SP6,SP7,SP8,⋯,SP11,SP12,SP13,SP14,SP15,SP16,SP17,SP18,SP19,SP20
,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,End-user,p1,7,6,5,5,5,6,4,5,⋯,4,5,5,5,4,4,4,5,3,3
2,End-user,p2,7,5,6,5,6,5,4,4,⋯,3,2,4,4,4,2,3,3,3,1
3,End-user,p3,7,6,6,5,6,5,4,4,⋯,2,3,4,4,4,3,3,3,3,2
4,End-user,p4,7,5,4,3,5,6,3,4,⋯,4,2,3,3,2,1,4,4,2,1
5,End-user,p5,6,6,5,4,6,6,5,4,⋯,5,4,6,5,6,4,5,6,5,5
6,End-user,p6,6,6,6,5,6,6,5,4,⋯,5,4,5,5,5,3,4,5,5,3


### **2. Reshaping the DB in four groups**

In [5]:
# 2. Reshaping the DB in four groups
plan_groups <- list(
  c("SP1", "SP5", "SP9", "SP13", "SP17"),
  c("SP2", "SP6", "SP10", "SP14", "SP18"),
  c("SP3", "SP7", "SP11", "SP15", "SP19"),
  c("SP4", "SP8", "SP12", "SP16", "SP20")
)

reshaped_dfs_list <- lapply(plan_groups, function(sp_cols) {
  df %>%
    # Select the participant info and the specific group's columns
    select(Particip, pid, all_of(sp_cols)) %>%
    # Rename the columns to generic group names for consistent stacking
    rename(
      group_1 = all_of(sp_cols[1]),
      group_2 = all_of(sp_cols[2]),
      group_3 = all_of(sp_cols[3]),
      group_4 = all_of(sp_cols[4]),
      group_5 = all_of(sp_cols[5])
    ) %>%
    # Create the `SPs_G` columns based on the original column names
    mutate(
      SPs_G1 = sp_cols[1],
      SPs_G2 = sp_cols[2],
      SPs_G3 = sp_cols[3],
      SPs_G4 = sp_cols[4],
      SPs_G5 = sp_cols[5]
    ) %>%
    # Reorder the columns to the final desired format
    select(
      Particip, pid,
      SPs_G1, group_1,
      SPs_G2, group_2,
      SPs_G3, group_3,
      SPs_G4, group_4,
      SPs_G5, group_5
    )
})
# Combining all four data frames from the list into a single one.
df_by_groups <- bind_rows(reshaped_dfs_list) %>%
  mutate(order = rep(c(4, 3, 2, 1), each = 77)) # Add the 'order' of SPs
print(tail(df_by_groups))

     Particip pid SPs_G1 group_1 SPs_G2 group_2 SPs_G3 group_3 SPs_G4 group_4
303 Test-user p72    SP4       5    SP8       4   SP12       2   SP16       2
304 Test-user p73    SP4       2    SP8       2   SP12       2   SP16       2
305 Test-user p74    SP4       3    SP8       2   SP12       2   SP16       4
306 Test-user p75    SP4       1    SP8       2   SP12       2   SP16       1
307 Test-user p76    SP4       4    SP8       4   SP12       3   SP16       2
308 Test-user p77    SP4       5    SP8       4   SP12       4   SP16       4
    SPs_G5 group_5 order
303   SP20       2     1
304   SP20       2     1
305   SP20       2     1
306   SP20       5     1
307   SP20       2     1
308   SP20       4     1


### **3. Create a function to perform the linear regression**

In [6]:
# 3. Create a function to perform the linear regression
analyze_group <- function(group_col) {
  formula_str <- paste(group_col, "~ order")
  lm_model <- lm(as.formula(formula_str), data = df_by_groups)
  lm_summary <- summary(lm_model)

  # Extract all the specific values to report
  intercept_est <- lm_summary$coefficients[1, "Estimate"]
  order_est <- lm_summary$coefficients[2, "Estimate"]
  std_error <- lm_summary$coefficients[2, "Std. Error"]
  r_squared <- lm_summary$r.squared
  f_statistic <- lm_summary$fstatistic[1]

  # Calculate the p-value for the F-statistic
  p_value <- pf(lm_summary$fstatistic[1], lm_summary$fstatistic[2], lm_summary$fstatistic[3], lower.tail = FALSE)

  # Create a data frame (a single row) with the results and return it
  results <- data.frame(
    Group = group_col,
    Intercept = intercept_est,
    Slope = order_est,
    p_value = p_value,
    R_squared = r_squared,
    Std_Error = std_error,
    F_stat = f_statistic,
    stringsAsFactors = FALSE
  )
  return(results)
}

# Loop to iterate over the group columns and call the function
group_columns <- c("group_1", "group_2", "group_3", "group_4", "group_5")

results_list <- list()
for (i in 1:length(group_columns)) {
  col <- group_columns[i]
  results_list[[i]] <- analyze_group(col)
}

# Combine all the individual data into a single table
final_results_table <- do.call(rbind, results_list)

### **4. Table 4. Linear Model Parameters of Participants’ Preferences for Plans**

In [7]:
# 4. Table 4. Linear Model Parameters of Participants’ Preferences for Plans
print("Table 4")
print(final_results_table, row.names = FALSE)   # Print the table


[1] "Table 4"
   Group Intercept     Slope      p_value R_squared  Std_Error    F_stat
 group_1  3.214286 0.8103896 2.229807e-36 0.4050515 0.05614586 208.33023
 group_2  2.837662 0.7649351 4.893257e-40 0.4367879 0.04965517 237.31222
 group_3  2.948052 0.6142857 1.609731e-23 0.2788891 0.05646704 118.34526
 group_4  2.480519 0.6077922 1.722847e-21 0.2567209 0.05912073 105.68923
 group_5  2.246753 0.5506494 4.363481e-16 0.1944618 0.06406789  73.87023
